In [17]:
# TO-DO
# PGLYCO ONLY HCD LEFT TO UNIQUEPEP FILE -> FROM UNIQUE HCD ALL IN / BYONIC ETD PASS CRITERIA ALL IN; BELOW CHECK PAIR -> _Quant 
# CHANGE COLOR CODE COLOR
# %%capture cap --no-stderr
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from IPython.display import display, HTML
import re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time
from datetime import datetime # attach current date to export filename
import sys
import ast # convert str back to tuple/list/int, etc

# INPUT EXCEL FILES
byonicfile = 'byonic_PTPRA_NoOG_T' # Please paste filename here. if None, please leave it as empty string '' (NOTE: DO NOT INCLUDE .XLSX HERE)
byosfile = '' # Please paste filename here. if None, please leave it as empty string '' (NOTE: DO NOT INCLUDE .XLSX HERE)
pglycofile = 'pglyco_PTPRA_NoOG_T' # Please paste filename here. if None, please leave it as empty string '' (NOTE: DO NOT INCLUDE .XLSX HERE)

# ADJUSTABLE PARAMETERS
# HCD: light colors
lightgreen_hex = '#ccffcc' 
lightblue_hex = '#cce6ff'
lightorange_hex = '#ffebcc'
# ETD: normal colors
normalgreen_hex = '#99ff99'
normalblue_hex = '#99ccff'
normalorange_hex = '#ffd699' 
# GlycanSource B+P, B/P: deep colors
darkgreen_hex = '#66ff66' 
darkblue_hex = '#66b3ff'
# byos colors
lightpink_hex = '#ffb6c1'
deeppink_hex = '#ff1493'
yellow_hex = '#ffff66' 
# BYONIC RT TOLERANCE
byonic_rt = 3
# PGLYCO RT TOLERANCE
pglyco_rt = 180

# AUXILIARY FUNCTIONS
def move_df(df, move_col, insert_left_col):
    move_df = df[move_col]
    df.drop(labels=[move_col], axis=1, inplace = True)
    df.insert(df.columns.get_loc(insert_left_col) + 1, move_col, move_df)
def sort_intstr_col(df, col_to_sort):
    # accepts int & str mixed type col, no other datatype is allowed
    lst = df[col_to_sort].tolist()
    lst.sort(key=lambda v: (isinstance(v, str), v))
    # reindex the df with .loc by the new sorted lst
    df = df.set_index(col_to_sort).loc[lst]
    df = df.drop_duplicates().reset_index()
    return df
# get export filename based on input combo
def export_filename(byonicfile = '', byosfile = '', pglycofile = ''):
    names = [byonicfile, byosfile, pglycofile]
    names = [n for n in names if n != '']
    names = '_'.join(names)
    return names
# define glycansource function
def glycansource(df, colname):
    # glycan comprison: only present in byonic -> b, only present in pglyco -> p, both the same -> b+p, not the same -> b/p
    conditions = [
        (df['Glycans[Byonic]'] != -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] == -1),
        (df['Glycans[Byonic]'] == -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] != -1),
        (df['Glycans[Byonic]'] != -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (df['Glycans[Byonic]'] == df['GlycanComposition_ByonicStyle[pGlyco]']),
        (df['Glycans[Byonic]'] != -1) & (df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (df['Glycans[Byonic]'] != df['GlycanComposition_ByonicStyle[pGlyco]'])]
    choices = ['B', 'P', 'B=P', 'B≠P'] 
    glycan_source = np.select(conditions, choices, -1) 
    df.insert(df.columns.get_loc(colname) + 1 , 'GlycanSource', glycan_source , True)
# define mask function
def threshold_masks_colorind(df):
    global b_hcd_mask, p_hcd_mask, both_hcd_mask, b_etd_mask, p_etd_mask, both_etd_mask
    global b_glycansource_mask, p_glycansource_mask, byos_exclusiveOr_mask, byos_and_mask, byos_bothsame_mask
    global lightgreen_ind, lightblue_ind, lightorange_ind, normalgreen_ind, normalblue_ind, normalorange_ind, deepgreen_ind, deepblue_ind, lightpink_ind, deeppink_ind, yellow_ind  

    if byonicfile != '' and byosfile != '' and pglycofile != '': # bbp all present
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['Score[Byonic]'] > 200) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4))
        p_hcd_mask = (df['FragmentType[pGlyco]'] == 'hcd') & ((df['Score[Byonic]'] <= 200) | (df['PEP\r\n2D[Byonic]'].abs() >= 0.001)) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['FragmentType[pGlyco]'] == 'hcd') & (df['Score[Byonic]'] > 200) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4)) 
        p_etd_mask = (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] == -1) & (df['PEP\r\n2D[Byonic]'] == -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # (HCD OR ETD) & (B+P OR B/P) & passes threshold: parts of lightblue/blue & lightgreen/green will become deep colors
        b_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((b_hcd_mask) ^ (b_etd_mask)) & (~((p_hcd_mask) ^ (p_etd_mask))) # hcd exclusive or etd & ~(p)
        p_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((p_hcd_mask) ^ (p_etd_mask)) & (~((b_hcd_mask) ^ (b_etd_mask))) # hcd exclusive or etd & ~(b)   
        # comparison between byonic & byos
        byos_exclusiveOr_mask = (df['Calc.M[Byos]'] != -1) & (df['Calc.M[Byonic]'] != -1) & ((df['Calc.M[Byos]'] != df['Calc.M[Byonic]'])^(df['PureSequence[Byos]'] != df['PureSequence[Byonic]']))
        byos_and_mask = (df['Calc.M[Byos]'] != -1) & (df['Calc.M[Byonic]'] != -1) & (df['Calc.M[Byos]'] != df['Calc.M[Byonic]']) & (df['PureSequence[Byos]'] != df['PureSequence[Byonic]'])
        byos_bothsame_mask = (df['Calc.M[Byos]'] != -1) & (df['Calc.M[Byonic]'] != -1) & (df['Calc.M[Byos]'] == df['Calc.M[Byonic]']) & (df['PureSequence[Byos]'] == df['PureSequence[Byonic]'])
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        lightblue_ind = df.loc[p_hcd_mask].index.tolist()
        df.loc[p_hcd_mask, 'ColorCode'] = 'lightblue(%s)'%lightblue_hex
        
        lightorange_ind = df.loc[both_hcd_mask].index.tolist()
        df.loc[both_hcd_mask, 'ColorCode'] = 'lightorange(%s)'%lightorange_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
        
        normalblue_ind = df.loc[p_etd_mask].index.tolist()
        df.loc[p_etd_mask, 'ColorCode'] = 'normalblue(%s)'%normalblue_hex
            
        normalorange_ind = df.loc[both_etd_mask].index.tolist()
        df.loc[both_etd_mask, 'ColorCode'] = 'normalorange(%s)'%normalorange_hex
        
        deepgreen_ind = df.loc[b_glycansource_mask].index.tolist()
        df.loc[b_glycansource_mask, 'ColorCode'] = 'deepgreen(%s)'%darkgreen_hex
        
        deepblue_ind = df.loc[p_glycansource_mask].index.tolist()
        df.loc[p_glycansource_mask, 'ColorCode'] = 'deepblue(%s)'%darkblue_hex
        
        lightpink_ind = df.loc[byos_exclusiveOr_mask].index.tolist()
        df.loc[byos_exclusiveOr_mask, 'ColorCode'] = 'lightpink(%s)'%lightpink_hex
        
        deeppink_ind = df.loc[byos_and_mask].index.tolist()
        df.loc[byos_and_mask, 'ColorCode'] = 'deeppink(%s)'%deeppink_hex
        
        yellow_ind = df.loc[byos_bothsame_mask].index.tolist()
        df.loc[byos_bothsame_mask, 'ColorCode'] = 'yellow(%s)'%yellow_hex
    elif byonicfile != '' and byosfile == '' and pglycofile != '': # byonic + pglyco
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['Score[Byonic]'] > 200) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4))
        p_hcd_mask = (df['FragmentType[pGlyco]'] == 'hcd') & ((df['Score[Byonic]'] <= 200) | (df['PEP\r\n2D[Byonic]'].abs() >= 0.001)) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['FragmentType[pGlyco]'] == 'hcd') & (df['Score[Byonic]'] > 200) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & ((df['PepScore[pGlyco]'] <= 5) | (df['GlyScore[pGlyco]'] <= 4)) 
        p_etd_mask = (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] == -1) & (df['PEP\r\n2D[Byonic]'] == -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)    
        both_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['FragmentType[pGlyco]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1) & (df['PepScore[pGlyco]'] > 5) & (df['GlyScore[pGlyco]'] > 4)
        # (HCD OR ETD) & (B+P OR B/P) & passes threshold: parts of lightblue/blue & lightgreen/green will become deep colors
        b_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((b_hcd_mask) ^ (b_etd_mask)) & (~((p_hcd_mask) ^ (p_etd_mask))) # hcd exclusive or etd & ~(p)
        p_glycansource_mask = ((df['GlycanSource'] == 'B=P') ^ (df['GlycanSource'] == 'B≠P')) & ((p_hcd_mask) ^ (p_etd_mask)) & (~((b_hcd_mask) ^ (b_etd_mask))) # hcd exclusive or etd & ~(b)   
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        lightblue_ind = df.loc[p_hcd_mask].index.tolist()
        df.loc[p_hcd_mask, 'ColorCode'] = 'lightblue(%s)'%lightblue_hex
        
        lightorange_ind = df.loc[both_hcd_mask].index.tolist()
        df.loc[both_hcd_mask, 'ColorCode'] = 'lightorange(%s)'%lightorange_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
        
        normalblue_ind = df.loc[p_etd_mask].index.tolist()
        df.loc[p_etd_mask, 'ColorCode'] = 'normalblue(%s)'%normalblue_hex
            
        normalorange_ind = df.loc[both_etd_mask].index.tolist()
        df.loc[both_etd_mask, 'ColorCode'] = 'normalorange(%s)'%normalorange_hex
        
        deepgreen_ind = df.loc[b_glycansource_mask].index.tolist()
        df.loc[b_glycansource_mask, 'ColorCode'] = 'deepgreen(%s)'%darkgreen_hex
        
        deepblue_ind = df.loc[p_glycansource_mask].index.tolist()
        df.loc[p_glycansource_mask, 'ColorCode'] = 'deepblue(%s)'%darkblue_hex
    elif byonicfile != '' and byosfile == '' and pglycofile == '': # only byonic
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType'] == 'hcd') & (df['Score'] > 200) & (df['PEP\r\n2D'].abs() < 0.001)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType'] == 'ethcd') & (df['Score'] != -1) & (df['PEP\r\n2D'] != -1) 
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
    elif byonicfile == '' and byosfile == '' and pglycofile != '': # only pglyco
        # HCD 
        p_hcd_mask = (df['FragmentType'] == 'hcd') & (df['PepScore'] > 5) & (df['GlyScore'] > 4)    
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        p_etd_mask = (df['FragmentType'] == 'ethcd') & (df['PepScore'] > 5) & (df['GlyScore'] > 4)    
        # record df color indices
        lightblue_ind = df.loc[p_hcd_mask].index.tolist()
        df.loc[p_hcd_mask, 'ColorCode'] = 'lightblue(%s)'%lightblue_hex
        
        normalblue_ind = df.loc[p_etd_mask].index.tolist()
        df.loc[p_etd_mask, 'ColorCode'] = 'normalblue(%s)'%normalblue_hex
    elif byonicfile != '' and byosfile != '' and pglycofile == '': # byonic + byos
        # HCD 
        b_hcd_mask = (df['Fragment\r\nType[Byonic]'] == 'hcd') & (df['Score[Byonic]'] > 200) & (df['PEP\r\n2D[Byonic]'].abs() < 0.001)
        # ETD: remember byonic etd does not need threshold, so we only need ot make sure that the row only contain byonic data, which means pglyco data will be -1
        b_etd_mask = (df['Fragment\r\nType[Byonic]'] == 'ethcd') & (df['Score[Byonic]'] != -1) & (df['PEP\r\n2D[Byonic]'] != -1)
        # comparison between byonic & byos
        byos_exclusiveOr_mask = (df['Calc.M[Byos]'] != -1) & (df['Calc.M[Byonic]'] != -1) & ((df['Calc.M[Byos]'] != df['Calc.M[Byonic]'])^(df['PureSequence[Byos]'] != df['PureSequence[Byonic]']))
        byos_and_mask = (df['Calc.M[Byos]'] != -1) & (df['Calc.M[Byonic]'] != -1) & (df['Calc.M[Byos]'] != df['Calc.M[Byonic]']) & (df['PureSequence[Byos]'] != df['PureSequence[Byonic]'])
        byos_bothsame_mask = (df['Calc.M[Byos]'] != -1) & (df['Calc.M[Byonic]'] != -1) & (df['Calc.M[Byos]'] == df['Calc.M[Byonic]']) & (df['PureSequence[Byos]'] == df['PureSequence[Byonic]'])
        # record df color indices
        lightgreen_ind = df.loc[b_hcd_mask].index.tolist()
        df.loc[b_hcd_mask, 'ColorCode'] = 'lightgreen(%s)'%lightgreen_hex
        
        normalgreen_ind = df.loc[b_etd_mask].index.tolist()
        df.loc[b_etd_mask, 'ColorCode'] = 'normalgreen(%s)'%normalgreen_hex
        
        lightpink_ind = df.loc[byos_exclusiveOr_mask].index.tolist()
        df.loc[byos_exclusiveOr_mask, 'ColorCode'] = 'lightpink(%s)'%lightpink_hex
        
        deeppink_ind = df.loc[byos_and_mask].index.tolist()
        df.loc[byos_and_mask, 'ColorCode'] = 'deeppink(%s)'%deeppink_hex
        
        yellow_ind = df.loc[byos_bothsame_mask].index.tolist()
        df.loc[byos_bothsame_mask, 'ColorCode'] = 'yellow(%s)'%yellow_hex
def bg_color(x):
    if byonicfile != '' and byosfile != '' and pglycofile != '': # bbp all present
        # byonic & pglyco colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex 
        c2 = 'background-color: %s'%lightblue_hex 
        c3 = 'background-color: %s'%lightorange_hex 
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex 
        c5 = 'background-color: %s'%normalblue_hex 
        c6 = 'background-color: %s'%normalorange_hex 
        # GlycanSource B+P, B/P: deep colors
        c7 = 'background-color: %s'%darkgreen_hex 
        c8 = 'background-color: %s'%darkblue_hex 
        # byos colors
        c9 = 'background-color: %s'%lightpink_hex 
        c10 = 'background-color: %s'%deeppink_hex
        c11 = 'background-color: %s'%yellow_hex 
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)    
        # byonic & byos coloring range
        bb_range = [col for col in df1.columns.tolist() if '[Byos]' in col]
        # byonic & pglyco coloring range
        bp_range = [col for col in df1.columns.tolist() if '[Byonic]' in col or '[pGlyco]' in col]
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, bp_range] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, bp_range]), lightgreen_hex))
        df1.loc[p_hcd_mask, bp_range] = c2 
        print('%s rows will be colored light blue (%s).'%(len(df1.loc[p_hcd_mask, bp_range]), lightblue_hex))
        df1.loc[both_hcd_mask, bp_range] = c3
        print('%s rows will be colored light orange (%s).'%(len(df1.loc[both_hcd_mask, bp_range]), lightorange_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, bp_range] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, bp_range]), normalgreen_hex))
        df1.loc[p_etd_mask, bp_range] = c5
        print('%s rows will be colored blue (%s).'%(len(df1.loc[p_etd_mask, bp_range]), normalblue_hex))
        df1.loc[both_etd_mask, bp_range] = c6
        print('%s rows will be colored orange (%s).'%(len(df1.loc[both_etd_mask, bp_range]), normalorange_hex))
        # GlycanSource B+P, B/P: deep colors (c7-c8)
        df1.loc[b_glycansource_mask, bp_range] = c7
        print('%s rows will be colored dark green (%s).'%(len(df1.loc[b_glycansource_mask, bp_range]), darkgreen_hex))
        df1.loc[p_glycansource_mask, bp_range] = c8
        print('%s rows will be colored dark blue (%s).'%(len(df1.loc[p_glycansource_mask, bp_range]), darkblue_hex))
        # byos colors (c9-c11)
        df1.loc[byos_exclusiveOr_mask, bb_range] = c9
        print('%s rows will be colored light pink (%s).'%(len(df1.loc[byos_exclusiveOr_mask, bb_range]), lightpink_hex))
        df1.loc[byos_and_mask, bb_range] = c10
        print('%s rows will be colored deep pink (%s).'%(len(df1.loc[byos_and_mask, bb_range]), deeppink_hex))
        df1.loc[byos_bothsame_mask, bb_range] = c11
        print('%s rows will be colored yellow (%s).'%(len(df1.loc[byos_bothsame_mask, bb_range]), yellow_hex))
        
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, bp_range]) + len(df1.loc[p_hcd_mask, bp_range]) \
                          + len(df1.loc[both_hcd_mask, bp_range]) + len(df1.loc[b_etd_mask, bp_range]) + len(df1.loc[p_etd_mask, bp_range]) \
                          + len(df1.loc[both_etd_mask, bp_range]))
        bb_white = len(df1) - (len(df1.loc[byos_exclusiveOr_mask, bb_range]) + len(df1.loc[byos_and_mask, bb_range]) + len(df1.loc[byos_bothsame_mask, bb_range]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        print('%s rows will be colorless in byos data (absent data in certain scans).'%bb_white)
        return df1
    elif byonicfile != '' and byosfile == '' and pglycofile != '': # byonic + pglyco
        # byonic & pglyco colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex 
        c2 = 'background-color: %s'%lightblue_hex 
        c3 = 'background-color: %s'%lightorange_hex 
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex 
        c5 = 'background-color: %s'%normalblue_hex 
        c6 = 'background-color: %s'%normalorange_hex 
        # GlycanSource B+P, B/P: deep colors
        c7 = 'background-color: %s'%darkgreen_hex 
        c8 = 'background-color: %s'%darkblue_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns) 
        # byonic & pglyco coloring range
        bp_range = [col for col in df1.columns.tolist() if '[Byonic]' in col or '[pGlyco]' in col]
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, bp_range] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, bp_range]), lightgreen_hex))
        df1.loc[p_hcd_mask, bp_range] = c2 
        print('%s rows will be colored light blue (%s).'%(len(df1.loc[p_hcd_mask, bp_range]), lightblue_hex))
        df1.loc[both_hcd_mask, bp_range] = c3
        print('%s rows will be colored light orange (%s).'%(len(df1.loc[both_hcd_mask, bp_range]), lightorange_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, bp_range] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, bp_range]), normalgreen_hex))
        df1.loc[p_etd_mask, bp_range] = c5
        print('%s rows will be colored blue (%s).'%(len(df1.loc[p_etd_mask, bp_range]), normalblue_hex))
        df1.loc[both_etd_mask, bp_range] = c6
        print('%s rows will be colored orange (%s).'%(len(df1.loc[both_etd_mask, bp_range]), normalorange_hex))
        # GlycanSource B+P, B/P: deep colors (c7-c8)
        df1.loc[b_glycansource_mask, bp_range] = c7
        print('%s rows will be colored dark green (%s).'%(len(df1.loc[b_glycansource_mask, bp_range]), darkgreen_hex))
        df1.loc[p_glycansource_mask, bp_range] = c8
        print('%s rows will be colored dark blue (%s).'%(len(df1.loc[p_glycansource_mask, bp_range]), darkblue_hex))
        
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, bp_range]) + len(df1.loc[p_hcd_mask, bp_range]) \
                          + len(df1.loc[both_hcd_mask, bp_range]) + len(df1.loc[b_etd_mask, bp_range]) + len(df1.loc[p_etd_mask, bp_range]) \
                          + len(df1.loc[both_etd_mask, bp_range]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        return df1
    elif byonicfile != '' and byosfile == '' and pglycofile == '': # only byonic
        # byonic colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns) 
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, :] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, :]), lightgreen_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, :] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, :]), normalgreen_hex))
        
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, :]) + len(df1.loc[b_etd_mask, :]))
        print('%s rows will be colorless in byonic data.'%bp_white)
        return df1
    elif byonicfile == '' and byosfile == '' and pglycofile != '': # only pglyco
        # pglyco colors
        # HCD: light colors
        c2 = 'background-color: %s'%lightblue_hex
        # ETD: normal colors
        c5 = 'background-color: %s'%normalblue_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns) 
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[p_hcd_mask, :] = c2 
        print('%s rows will be colored light blue (%s).'%(len(df1.loc[p_hcd_mask, :]), lightblue_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[p_etd_mask, :] = c5
        print('%s rows will be colored blue (%s).'%(len(df1.loc[p_etd_mask, :]), normalblue_hex))
        
        bp_white = len(df1) - (len(df1.loc[p_hcd_mask, :]) + len(df1.loc[p_etd_mask, :]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        return df1
    elif byonicfile != '' and byosfile != '' and pglycofile == '': # byonic + byos
        # byonic colors
        # HCD: light colors
        c1 = 'background-color: %s'%lightgreen_hex
        # ETD: normal colors
        c4 = 'background-color: %s'%normalgreen_hex
        # byos colors
        c9 = 'background-color: %s'%lightpink_hex 
        c10 = 'background-color: %s'%deeppink_hex
        c11 = 'background-color: %s'%yellow_hex
        c = '' 
        #DataFrame with same index and columns names as original filled empty strings
        df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)    
        # byonic & byos coloring range
        bb_range = [col for col in df1.columns.tolist() if '[Byos]' in col]
        # byonic & pglyco coloring range
        bp_range = [col for col in df1.columns.tolist() if '[Byonic]' in col]
        # modify values of df1 column by boolean mask
        # HCD: light colors (c1-c3)
        df1.loc[b_hcd_mask, bp_range] = c1
        print('<Color Summary>\n%s rows will be colored light green (%s).'%(len(df1.loc[b_hcd_mask, bp_range]), lightgreen_hex))
        # ETD: normal colors (c4-c6)
        df1.loc[b_etd_mask, bp_range] = c4
        print('%s rows will be colored green (%s).'%(len(df1.loc[b_etd_mask, bp_range]), normalgreen_hex))
        # byos colors (c9-c11)
        df1.loc[byos_exclusiveOr_mask, bb_range] = c9
        print('%s rows will be colored light pink (%s).'%(len(df1.loc[byos_exclusiveOr_mask, bb_range]), lightpink_hex))
        df1.loc[byos_and_mask, bb_range] = c10
        print('%s rows will be colored deep pink (%s).'%(len(df1.loc[byos_and_mask, bb_range]), deeppink_hex))
        df1.loc[byos_bothsame_mask, bb_range] = c11
        print('%s rows will be colored yellow (%s).'%(len(df1.loc[byos_bothsame_mask, bb_range]), yellow_hex))
        bp_white = len(df1) - (len(df1.loc[b_hcd_mask, bp_range]) + len(df1.loc[b_etd_mask, bp_range]))
        bb_white = len(df1) - (len(df1.loc[byos_exclusiveOr_mask, bb_range]) + len(df1.loc[byos_and_mask, bb_range]) + len(df1.loc[byos_bothsame_mask, bb_range]))
        print('%s rows will be colorless in byonic & pglyco data.'%bp_white)
        print('%s rows will be colorless in byos data (absent data in certain scans).'%bb_white)
        return df1

# PREPROCESSING FUNCTIONS
def pglyco_preprocessor(pglycofile):
    if pglycofile != '':
        # read in pglyco as df
        pglyco_df = pd.read_excel('%s.xlsx'%pglycofile, header = 0)
        pglyco_df = pglyco_df.fillna('N/A')
        pglyco_df = pglyco_df.sort_values(by=['Scan'])
        pglyco_df = pglyco_df.reset_index(drop = True)

        # output column name
        print('Original pglyco columns:\n%s\n'%pglyco_df.columns)

        # replace _x000D_ w/ \r if exists
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in pglyco_df.columns]
        pglyco_df.columns = fixed_colname
        print('Fixed pglyco columns:\n%s\n'%pglyco_df.columns)

        # record original data size
        print('Original pglyco data size:\nrow: %s\ncol: %s\n'%(pglyco_df.shape[0], pglyco_df.shape[1]))

        # change J back to N as column named peptide(J-->N)
        pglyco_df['Peptide'] = pglyco_df['Peptide'].str.replace('J','N')

        # analyze sequon in pglyco file
        pglyco_sequon = pglyco_df['Peptide'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
        pglyco_sequon_lst = []
        for t in pglyco_sequon:
            t = str(t)
            res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
            res = ''.join(res)
            res = textwrap.wrap(res, 3)
            if res == []:
                pglyco_sequon_lst.append('N/A')
            elif len(res) == 1:
                pglyco_sequon_lst.append(res[0])
            else:
                pglyco_sequon_lst.append(res)

        pglyco_sequon_lst = [tuple(i) if type(i) == list else i for i in pglyco_sequon_lst]
        pglyco_df.insert(pglyco_df.columns.get_loc('Peptide') + 1 , 'Sequon', pglyco_sequon_lst , True)

        # replace H, N, A, F, G symbols w/ Hex, HexNAc, NeuAc, Fuc, NeuGc
        byonicstyle = []
        if 'Glycan(H,N,A,G,F)' in pglyco_df.columns:
            glycan_num = pglyco_df['Glycan(H,N,A,G,F)'].tolist()
            for i in glycan_num:
                i = i.split(' ')
                new_order = [1, 0, 4, 2, 3]
                i = [i[x] for x in new_order]
                if i[0] != '0':
                    n = 'HexNAc(%s)'%(i[0])
                else :
                    n = ''
                if i[1] != '0':
                    h = 'Hex(%s)'%(i[1])
                else:
                    h = ''
                if i[2] != '0':
                    f = 'Fuc(%s)'%(i[2])
                else:
                    f = ''
                if i[3] != '0':
                    a = 'NeuAc(%s)'%(i[3])
                else:
                    a = ''
                if i[4] != '0':
                    g = 'NeuGc(%s)'%(i[4])
                else:
                    g = ''
                each_byonicstyle = n + h + f + a + g
                byonicstyle.append(each_byonicstyle)
        elif 'Glycan(H,N,A,F)' in pglyco_df.columns:
            glycan_num = pglyco_df['Glycan(H,N,A,F)'].tolist()
            for i in glycan_num:
                i = i.split(' ')
                new_order = [1, 0, 3, 2]
                i = [i[x] for x in new_order]
                if i[0] != '0':
                    n = 'HexNAc(%s)'%(i[0])
                else :
                    n = ''
                if i[1] != '0':
                    h = 'Hex(%s)'%(i[1])
                else:
                    h = ''
                if i[2] != '0':
                    f = 'Fuc(%s)'%(i[2])
                else:
                    f = ''
                if i[3] != '0':
                    a = 'NeuAc(%s)'%(i[3])
                else:
                    a = ''
                each_byonicstyle = n + h + f + a
                byonicstyle.append(each_byonicstyle)
        elif 'Glycan(H,N,F,A)' in pglyco_df.columns:
            glycan_num = pglyco_df['Glycan(H,N,F,A)'].tolist()
            for i in glycan_num:
                i = i.split(' ')
                new_order = [1, 0, 2, 3]
                i = [i[x] for x in new_order]
                if i[0] != '0':
                    n = 'HexNAc(%s)'%(i[0])
                else :
                    n = ''
                if i[1] != '0':
                    h = 'Hex(%s)'%(i[1])
                else:
                    h = ''
                if i[2] != '0':
                    f = 'Fuc(%s)'%(i[2])
                else:
                    f = ''
                if i[3] != '0':
                    a = 'NeuAc(%s)'%(i[3])
                else:
                    a = ''
                each_byonicstyle = n + h + f + a
                byonicstyle.append(each_byonicstyle)
        else:
            sys.exit('Please check if there are glycans other than H, N, A, G, F or if column name has changed. This app will stop.')
        pglyco_df.insert(pglyco_df.columns.get_loc('GlycanComposition') + 1 , 'GlycanComposition_ByonicStyle', byonicstyle , True)

        # if the etdscan is not blank, duplicate the row and change the duplicated scan to etdscan (micmic byonic format)
        pglyco_df.insert(pglyco_df.columns.get_loc('Scan') + 1 , 'FragmentType', 'hcd' , True) # insert 'fragment type' col
        row_to_duplicate = pglyco_df[pglyco_df['ETDScan'] != 'N/A'].copy() # assume the missing ETDScan in pglyco is represented as blank & later filled w/ N/A
        row_to_duplicate['FragmentType'] = 'ethcd'
        row_to_duplicate['Scan'] = row_to_duplicate['ETDScan']
        pglyco_df = pd.concat([pglyco_df, row_to_duplicate]) # duplicate w/ index
        pglyco_df = pglyco_df.sort_values(by = ['Scan']) # sort by scan directly, the duplicated ethcd rows can be separated from hcd rows
        print('----- pGlyco data preprocessing completed. -----\n')
        return pglyco_df

def byonic_preprocessor(byonicfile):
    if byonicfile != '':
        byonic_df = pd.read_excel('%s.xlsx'%byonicfile, header = 0)
        byonic_df = byonic_df.fillna('N/A')
        print('Original byonic columns:\n%s\n'%byonic_df.columns)
        # replace _x000D_ w/ \r if exists
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in byonic_df.columns]
        byonic_df.columns = fixed_colname
        print('Fixed byonic columns:\n%s\n'%byonic_df.columns)
        # record original data size
        print('Original byonic data size:\nrow: %s\ncol: %s\n'%(byonic_df.shape[0], byonic_df.shape[1]))
        # extract 'scan' from 'scan #' in byonic file & add a 'Scan' column
        byonic_scan = byonic_df['Scan #'].tolist()
        byonic_scan_lst = []
        for scan in byonic_scan:
            scan = scan.split(' ')[-1].split('=')[-1]
            scan = int(scan)
            byonic_scan_lst.append(scan)
        byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Scan', byonic_scan_lst , True)
        byonic_df = byonic_df.sort_values(by = ['Scan'])
        byonic_df = byonic_df.reset_index(drop = True)
        # add 'PureSequence' column to byonic file
        if 'Sequence\r\n(unformatted)' in byonic_df.columns: # deal w/ dif byonic version
            byonic_seq = byonic_df['Sequence\r\n(unformatted)'].str[2:-2].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()
            byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
            byonic_df.insert(byonic_df.columns.get_loc('Sequence\r\n(unformatted)') + 1 , 'PureSequence', byonic_seq , True)
            byonic_seq_forSequonNsite = byonic_df['Sequence\r\n(unformatted)'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()
            byonic_seq_forSequonNsite = [''.join(each_pure) for each_pure in byonic_seq_forSequonNsite]
            byonic_seq_forSequonNsite = pd.Series(byonic_seq_forSequonNsite) # preserve aa outside dots
        elif 'Sequence' in byonic_df.columns:
            byonic_seq = byonic_df['Sequence'].str[2:-2].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()  
            byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
            byonic_df.insert(byonic_df.columns.get_loc('Sequence') + 1 , 'PureSequence', byonic_seq , True)
            byonic_seq_forSequonNsite = byonic_df['Sequence'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()
            byonic_seq_forSequonNsite = [''.join(each_pure) for each_pure in byonic_seq_forSequonNsite]
            byonic_seq_forSequonNsite = pd.Series(byonic_seq_forSequonNsite)
        else:
            sys.exit('Please check if sequence column name has changed. This app will stop.')  
        # add 'Sequon' & 'N-site' column to byonic file
        byonic_sequon_lst = []
        n_site_lst = []
        pureseq_cnt = 0 # this is byonic_seq_forSequonNsite
        pos = byonic_df['Pos.'].tolist()
        pos_cnt = 0
        byonic_sequon = byonic_seq_forSequonNsite.str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
        for t in byonic_sequon:
            t = str(t)
            res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
            res = ''.join(res)
            res = textwrap.wrap(res, 3)
            if res == []:
                byonic_sequon_lst.append('N/A')
                n_site_lst.append('N/A')
                pureseq_cnt += 1
                pos_cnt += 1
            elif len(res) == 1: # single sequon -> single n-site
                byonic_sequon_lst.append(res[0])
                regex = re.compile(r'(' + '|'.join(res) + r')')
                each_n_site_pos = [m.start() for m in regex.finditer(byonic_seq_forSequonNsite[pureseq_cnt])][0]
                each_n_site = pos[pos_cnt] -1 + each_n_site_pos # -1 to fix the 'dots problem'
                n_site_lst.append(each_n_site)
                pureseq_cnt += 1
                pos_cnt += 1
            else: # non-single sequon
                byonic_sequon_lst.append(res)
                regex = re.compile(r'(' + '|'.join(res) + r')')
                each_n_site_pos = [m.start() for m in regex.finditer(byonic_seq_forSequonNsite[pureseq_cnt])]
                each_n_site = list(pos[pos_cnt] -1 + np.array(each_n_site_pos))
                n_site_lst.append(each_n_site)
                pureseq_cnt += 1
                pos_cnt += 1
        # convert lists within list to tuple for later usage ('cos lists are unhashable, which may cause some problems later)
        byonic_sequon_lst = [tuple(i) if type(i) == list else i for i in byonic_sequon_lst]
        n_site_lst = [tuple(i) if type(i) == list else i for i in n_site_lst]
        byonic_df.insert(byonic_df.columns.get_loc('PureSequence') + 1 , 'Sequon', byonic_sequon_lst , True)
        byonic_df.insert(0, 'N-site(SequonBased)', n_site_lst , True)
        # ONLY FOR HCD//ETD FILES: add N/A col 'Pair' for later modification (pair: cal.m/z same, cal.m same, pureseq same, scan difference <= 5, hcd before ethcd)
        if 'ethcd' in byonic_df['Fragment\r\nType'].tolist():
            byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Pair', 'N/A' , True)
            # potential_pair = byonic_df[['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence']][byonic_df.duplicated(subset=['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence'], keep=False)]
            potential_pair = byonic_df[byonic_df.duplicated(subset=['Calc.\r\nm/z', 'Calc.M', 'PureSequence'], keep=False)].sort_values(['Calc.\r\nm/z'])
            # print(potential_pair)
            mz_gp = sorted(list(set(potential_pair['Calc.\r\nm/z'].tolist())))
            pair_cnt = 0
            all_hcd_ind = []
            all_etd_ind = []
            for i in mz_gp:
                gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
                gp = gp.sort_values(['Scan'])
                # skip the gp w/o ethcd & find pairs: last criterion -> scan dif <= 5
                if 'hcd' in gp['Fragment\r\nType'].tolist() and 'ethcd' in gp['Fragment\r\nType'].tolist():
                    # from hcd find the nearest ethcd below
                    pair_candidate = gp[(gp['Fragment\r\nType'] == 'hcd') & (gp['Fragment\r\nType'].shift(-1) == 'ethcd') & (gp['Scan'].shift(-1) - gp['Scan'] <= 5)]
                    hcd_iloc = [gp.index.get_loc(ind) for ind in pair_candidate.index] # get the positions of the hcd in pairs in gp df
                    etd_iloc = list(np.array(hcd_iloc) + 1)
                    hcd_ind = pair_candidate.index.tolist()
                    etd_ind = [gp.index[i] for i in etd_iloc] 
                    all_hcd_ind.extend(hcd_ind)
                    all_etd_ind.extend(etd_ind)
                else:
                    pass
            all_hcd_ind.sort()
            all_etd_ind.sort()
            byonic_df['Pair'].iloc[all_hcd_ind] = ['pair%s'%(i+1) for i in range(len(all_hcd_ind))]
            byonic_df['Pair'].iloc[all_etd_ind] = ['pair%s'%(i+1) for i in range(len(all_etd_ind))]
        else:
            pass
        print('----- Byonic data preprocessing completed. -----\n')
        return byonic_df

def byos_preprocessor(byosfile):
    if byosfile != '':
        byos_df = pd.read_excel('%s.xlsx'%byosfile, header = 0)
        # output column name
        print('Original byos columns:\n%s\n'%byos_df.columns)
        # replace _x000D_ w/ \r if exists
        fixed_colname = [i.replace('_x000D_', '\r') if '_x000D_' in i else i for i in byos_df.columns]
        byos_df.columns = fixed_colname
        print('Fixed byos columns:\n%s\n'%byos_df.columns)
        # extract needed columns: Scan Number(s)(Posit), MS Alias name, XIC area summed, XIC AUC, Apex Int.(Posit), Calc.M, Sequence
        byos_df = byos_df[['Scan Number(s)\r\n(Posit)', 'Glycans', 'MS2 Search\r\nAlias name', 'XIC area\r\nsummed', 'XIC\r\nAUC', 'Apex Int.\r\n(Posit)', 'Calc.M', 'Sequence']]
        byos_df = byos_df.fillna('N/A')
        # output extracted column name
        print('Extracted byos columns:\n%s\n'%byos_df.columns)
        # drop the row w/ multiple scan numbers (data type would be str)
        byos_df = byos_df[byos_df['Scan Number(s)\r\n(Posit)'].apply(lambda x: isinstance(x, int))]
        # sort byos by scan
        byos_df = byos_df.sort_values(by=['Scan Number(s)\r\n(Posit)'])
        byos_df = byos_df.reset_index(drop = True)
        # add 'PureSequence' col
        byos_df['Sequence'] = byos_df['Sequence'].str.upper()
        byos_df['PureSequence'] = byos_df['Sequence'].str[2:-2]
        # display(HTML(byos_df.to_html()))
        print('----- Byos data preprocessing completed. -----\n')
        return byos_df

def main():
    # TIMESTAMP
    start_time = time.time()
    # CURRENT DATE
    date = datetime.now().strftime('%Y%m%d')
    # PREPARE EXPORT FILENAME
    filename = export_filename(byonicfile, byosfile, pglycofile)
    # FILE PREPROCESSING
    byonic_df = byonic_preprocessor(byonicfile)
    byos_df =  byos_preprocessor(byosfile)
    pglyco_df = pglyco_preprocessor(pglycofile)
    # INPUT COMBO CONTROL FLOW
    if byonicfile != '' and byosfile != '' and pglycofile != '': # bbp all present
        print('----- Start combining Byonic & Byos & pGlyco. -----\n')
        # combined data based on 'Scan'
        byonic_scanasid = byonic_df.copy()
        new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
        byonic_scanasid.columns = new_byonic_col
        pglyco_scanasid = pglyco_df.copy()
        new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
        pglyco_scanasid.columns = new_pglyco_col
        byos_scanasid = byos_df.copy()
        new_byos_col = [n + '[Byos]' if n != 'Scan Number(s)\r\n(Posit)' else n for n in byos_scanasid.columns]
        byos_scanasid.columns = new_byos_col
        byonic_scanasid = byonic_scanasid.set_index('Scan')
        pglyco_scanasid = pglyco_scanasid.set_index('Scan')
        byos_scanasid = byos_scanasid.set_index('Scan Number(s)\r\n(Posit)')
        # align scan & concat (all align on row to make row number all the same)
        a1, a2 = byonic_scanasid.align(pglyco_scanasid, join = 'outer', axis = 0) # row: a1 = a2
        a1, a3 = a1.align(byos_scanasid, join = 'outer', axis = 0) # row: a1 = a2 = a3
        a2, a3 = a2.align(byos_scanasid, join = 'outer', axis = 0) # row: a2 = a1 = a3
        all_combined_df = pd.concat([a1,a3,a2], axis = 1)
        all_combined_df.index.name = 'Scan'
        all_combined_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        all_combined_df = all_combined_df.fillna(-1)
        move_df(all_combined_df, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        ## result post-processing 
        glycansource(all_combined_df, 'Scan')
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        # analyze PSM (need to pass threshold)
        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind + lightorange_ind + normalorange_ind + deepgreen_ind)) # byonic pass threshold
        byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_colored_id]
        pglyco_colored_id = list(set(lightblue_ind + normalblue_ind + lightorange_ind + normalorange_ind + deepblue_ind)) # pglyco pass threshold
        pglyco_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in pglyco_colored_id]
        # using groupby size function to count psm & add psm columns (new criterion: RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif)
        # byonic & byos
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)[Byonic]': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract[Byonic]'] = all_combined_df.loc[all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]'])['Score[Byonic]'].idxmax(), 'Scan\r\nTime[Byonic]'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna[Byonic]'] = all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]'])['to_subtract[Byonic]'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif[Byonic]'] = (all_combined_df['Scan\r\nTime[Byonic]'] - all_combined_df['to_subtract_fillna[Byonic]']).abs() # get absolute dif
        all_combined_df['rt_dif_reset[Byonic]'] = [0 if dif < byonic_rt else dif for dif in all_combined_df['rt_dif[Byonic]']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        byonic_psm = all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset[Byonic]'])['N-site(SequonBased)[Byonic]'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('N-site(SequonBased)[Byonic]') + 1 , 'PSM[Byonic]', byonic_psm , True)
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == 'N/A'), 'PSM[Byonic]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM[Byonic]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == '-1'), 'PSM[Byonic]'] = -1 # if site is '-1', then psm set to -1
        # pglyco
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'int'})
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract[pGlyco]'] = all_combined_df.loc[all_combined_df.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'])['GlyScore[pGlyco]'].idxmax(), 'RT[pGlyco]'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna[pGlyco]'] = all_combined_df.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'])['to_subtract[pGlyco]'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif[pGlyco]'] = (all_combined_df['RT[pGlyco]'] - all_combined_df['to_subtract_fillna[pGlyco]']).abs() # get absolute dif
        all_combined_df['rt_dif_reset[pGlyco]'] = [0 if dif < pglyco_rt else dif for dif in all_combined_df['rt_dif[pGlyco]']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        pglyco_psm = all_combined_df.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset[pGlyco]'])['ProSites[pGlyco]'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('ProSites[pGlyco]') + 1 , 'PSM[pGlyco]', pglyco_psm , True)
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == 'N/A'), 'PSM[pGlyco]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[pglyco_belowthreshold_id, 'PSM[pGlyco]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == '-1'), 'PSM[pGlyco]'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['to_subtract[Byonic]', 'to_subtract_fillna[Byonic]', 'rt_dif[Byonic]', 'rt_dif_reset[Byonic]', 'to_subtract[pGlyco]', 'to_subtract_fillna[pGlyco]', 'rt_dif[pGlyco]', 'rt_dif_reset[pGlyco]'], axis = 1) # drop the intermediate cols

        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_All.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        if 'Pair[Byonic]' in all_combined_df.columns.tolist():
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
            pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_Pair.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[Byonic]' in col or '[Byos]' in col]
        pglyco_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[pGlyco]' in col]
        simple_byonicbyos = all_forsimple_df.loc[byonic_colored_id, byonicbyos_col_range] # extract colored (pass threshold) rows
        simple_byonicbyos = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # for simplified, n-site must exist
        simple_pglyco = all_forsimple_df.loc[pglyco_colored_id, pglyco_col_range] # extract colored (pass threshold) rows
        simple_pglyco = simple_pglyco[(simple_pglyco['ProSites[pGlyco]'] != 'N/A')&(simple_pglyco['ProSites[pGlyco]'] != '-1')] # for simplified, n-site must exist
        simple_pglyco = simple_pglyco.loc[simple_pglyco['FragmentType[pGlyco]'] == 'hcd'] # pglyco etd has the same values as hcd, so only let hcd enter unique file
        # get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos = simple_byonicbyos.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_pglyco = simple_pglyco.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['GlyScore[pGlyco]'].idxmax()]).reset_index(drop=True)

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan
        simple_byonicbyos = simple_byonicbyos.set_index('Scan')
        simple_pglyco = simple_pglyco.set_index('Scan')
        a1, a2 = simple_byonicbyos.align(simple_pglyco, join = 'outer', axis = 0) # row: a1 = a2
        id_df = pd.concat([a1,a2], axis = 1)
        id_df.index.name = 'Scan'
        id_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df = id_df.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df['PepMS[Byonic]'] = id_df['Calc.\r\nMH[Byonic]'] - id_df['glycan_mods_sum[Byonic]']
        id_df.loc[(id_df['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(id_df, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
        id_df['ProSites[pGlyco]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['ProSites[pGlyco]'].tolist()] # convert str back to int & tuple
        
        id_df.loc[(id_df['N-site(SequonBased)[Byonic]'] == -1), 'mock_site[Byonic]'] = id_df['ProSites[pGlyco]']
        id_df.loc[(id_df['N-site(SequonBased)[Byonic]'] != -1), 'mock_site[Byonic]'] = id_df['N-site(SequonBased)[Byonic]']
        # drop intermediate cols
        id_df = id_df.drop(['to_subtract[Byonic]', 'to_subtract_fillna[Byonic]', 'rt_dif[Byonic]', 'rt_dif_reset[Byonic]', 'to_subtract[pGlyco]', 'to_subtract_fillna[pGlyco]', 'rt_dif[pGlyco]', 'rt_dif_reset[pGlyco]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        glycansource(id_df, 'Scan')
        move_df(id_df, 'mock_site[Byonic]', 'GlycanSource')
        col_order = id_df.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df = sort_intstr_col(id_df, 'Charge[pGlyco]')
        print('Done sorting by Charge[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PrecursorMH[pGlyco]')
        print('Done sorting by PrecursorMH[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PeptideMH[pGlyco]')
        print('Done sorting by PeptideMH[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'GlycanComposition_ByonicStyle[pGlyco]')
        print('Done sorting by GlycanComposition_ByonicStyle[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Peptide[pGlyco]')
        print('Done sorting by Peptide[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Glycans[Byonic]')
        print('Done sorting by Glycans[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'mock_site[Byonic]')
        print('Done sorting by mock_site[Byonic] (added during processing)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df['Glycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df['Glycans[Byonic]']]
        id_df['GlycanComposition_ByonicStyle[pGlyco]'] = [int(i) if i == '-1' else i for i in id_df['GlycanComposition_ByonicStyle[pGlyco]']]
        id_df = id_df[col_order]
        threshold_masks_colorind(id_df)
        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        id_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_UniquePep.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')
        print('----- Start preparing quantification file. -----\n')

        # quantification only apply to single sites (quant_df offers important info. for later multiIndex df construction)
        quant = id_df[id_df['N-site(SequonBased)[Byonic]'].apply(lambda x: isinstance(x, int))]
        drop_ind = quant.loc[(quant['Fragment\r\nType[Byonic]'] == 'ethcd') & ((quant['Score[Byonic]'] <= 200)|(quant['PEP\r\n2D[Byonic]'].abs() >= 0.001)) & ((quant['Pair[Byonic]'] == -1)|(quant['Pair[Byonic]'] == 'N/A'))].index.tolist()
        ind = [i for i in quant.index.tolist() if i not in drop_ind]
        quant = quant.loc[ind] # hcd all in, if byonic etd pass score & pep2d all in, otherwise in with pair
        # normalize by sum of xicauc/ int/ mono/ iso respectively
        total_xicauc = quant.loc[quant['XIC\r\nAUC[Byos]'] >= 0, 'XIC\r\nAUC[Byos]'].sum()
        print('Total XIC AUC[Byos]:%s'%total_xicauc)
        total_int = quant.loc[quant['Apex Int.\r\n(Posit)[Byos]'] >= 0, 'Apex Int.\r\n(Posit)[Byos]'].sum()
        print('Total Apex Int. (Posit)[Byos]:%s'%total_int)
        total_mono = quant.loc[quant['MonoArea[pGlyco]'] >= 0, 'MonoArea[pGlyco]'].sum()
        print('Total MonoArea[pGlyco]:%s'%total_mono)
        total_iso = quant.loc[quant['IsotopeArea[pGlyco]'] >= 0, 'IsotopeArea[pGlyco]'].sum()
        print('Total IsotopeArea[pGlyco]:%s'%total_iso)
        # avoid adding values from below-threshold rows: if deepgreen -> change value to -1, if deepblue -> change value to -1.(-1 to make lambda function easier to write)
        threshold_masks_colorind(quant)
        quant.loc[deepgreen_ind, ['MonoArea[pGlyco]', 'IsotopeArea[pGlyco]']] = -1
        quant.loc[deepblue_ind, ['XIC\r\nAUC[Byos]', 'Apex Int.\r\n(Posit)[Byos]']] = -1
        # convert all the 'N/A' in byonic glycans to 'Unoccupied'
        quant.loc[(quant['Glycans[Byonic]'] == 'N/A'), ['Glycans[Byonic]']] = 'Unoccupied'
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        quant['e_sum_XIC\r\nAUC[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['f_sum_Apex Int.\r\n(Posit)[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['g_sum_MonoArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['h_sum_IsotopeArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['a_norm_XIC\r\nAUC[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x != -1].sum()/total_xicauc)
        quant['b_norm_Apex Int.\r\n(Posit)[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x != -1].sum()/total_int)
        quant['c_norm_MonoArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x != -1].sum()/total_mono)
        quant['d_norm_IsotopeArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x != -1].sum()/total_iso)
        # extract the needed cols only & split byonicbyos/pglyco, drop all -1 rows, glycans as index to outer union concat data again
        quant_bb = quant[['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]', 'MS2 Search\r\nAlias name[Byos]', 'XIC\r\nAUC[Byos]', 'Apex Int.\r\n(Posit)[Byos]', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
        quant_p = quant[['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]', 'MonoArea[pGlyco]', 'IsotopeArea[pGlyco]', 'g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
        quant_bb = quant_bb.loc[(quant_bb['N-site(SequonBased)[Byonic]'] != -1) & (quant_bb['MS2 Search\r\nAlias name[Byos]'] != -1)]
        quant_p = quant_p.loc[quant_p['ProSites[pGlyco]'] != -1]
        quant_bb = quant_bb.set_index('Glycans[Byonic]')
        quant_p = quant_p.set_index('GlycanComposition_ByonicStyle[pGlyco]')
        a1, a2 = quant_bb.align(quant_p, join = 'outer', axis = 0) 
        quant_glycanid = pd.concat([a1,a2], axis = 1)
        quant_glycanid.index.name = 'Glycans ↓'
        quant_glycanid.reset_index(level=0, inplace=True)
        # add N(x) col
        nx = ['Unoccupied' if lst == [] else 'N(%s)'%(lst[0]) for lst in quant_glycanid['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        quant_glycanid.insert(0 , 'N(x) ↓', nx , True)
        move_df(quant_glycanid, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        # change all nan to blank -1
        quant_glycanid = quant_glycanid.fillna(-1)
        # quant_glycanid.to_excel('quant_glycanid.xlsx', index = False)
        print('----- Start summary table construction. -----\n')
        # start grouping: N(X) -> byonic glycans -> byonic sites -> .agg({all 8 cols, apply .mean() & .mean()/total XXX})
        quant = quant_glycanid.groupby(['N(x) ↓', 'Glycans ↓', 'N-site(SequonBased)[Byonic]', 'ProSites[pGlyco]']).agg({'e_sum_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'f_sum_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean() \
                                                                                              , 'g_sum_MonoArea[pGlyco]': lambda x: x.mean(), 'h_sum_IsotopeArea[pGlyco]': lambda x: x.mean() \
                                                                                              , 'a_norm_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x.mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x.mean()})


        # fix the str '10' sorted before '2' problem
        nx = list(set(nx))
        nx  = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str and i != 'Unoccupied' else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]
        quant = quant.reindex(new_nx, level = 0)
        # unstack ms2 alias name, byonic n-sites, pglyco n-sites
        quant = quant.unstack(level=-1)
        quant = quant.unstack(level=-1)
        # multiIndex col swaplevel
        quant.columns = quant.columns.swaplevel(0, 2)
        quant.columns = quant.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        quant.sort_index(axis=1, level=[0, 1, 2], inplace=True)
        quant.columns = quant.columns.set_names(['ProSites[pGlyco] →', 'N-site(SequonBased)[Byonic] →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        quant.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]', 'sum_XIC\r\nAUC[Byos]', 'sum_Apex Int.\r\n(Posit)[Byos]', 'sum_MonoArea[pGlyco]', 'sum_IsotopeArea[pGlyco]'], level = 2, inplace=True)
        print('\n----- Exporting "_Quant" file... This may take some time, please wait. -----\n')
        quant.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_Quant.xlsx', index = False)  
        print('\n----- "_Quant" file exported. -----\n')
    elif byonicfile != '' and byosfile == '' and pglycofile != '': # byonic + pglyco 
        print('----- Start combining Byonic & pGlyco. -----\n')
        # combined data based on 'Scan'
        byonic_scanasid = byonic_df.copy()
        new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
        byonic_scanasid.columns = new_byonic_col
        pglyco_scanasid = pglyco_df.copy()
        new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
        pglyco_scanasid.columns = new_pglyco_col
        byonic_scanasid = byonic_scanasid.set_index('Scan')
        pglyco_scanasid = pglyco_scanasid.set_index('Scan')
        # align scan & concat (all align on row to make row number all the same)
        a1, a2 = byonic_scanasid.align(pglyco_scanasid, join = 'outer', axis = 0) # row: a1 = a2
        all_combined_df = pd.concat([a1,a2], axis = 1)
        all_combined_df.index.name = 'Scan'
        all_combined_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        all_combined_df = all_combined_df.fillna(-1)
        move_df(all_combined_df, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        ## result post-processing 
        glycansource(all_combined_df, 'Scan')
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        # analyze PSM (need to pass threshold)
        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind + lightorange_ind + normalorange_ind + deepgreen_ind)) # byonic pass threshold
        byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_colored_id]
        pglyco_colored_id = list(set(lightblue_ind + normalblue_ind + lightorange_ind + normalorange_ind + deepblue_ind)) # pglyco pass threshold
        pglyco_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in pglyco_colored_id]
        # using groupby size function to count psm & add psm columns (new criterion: RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif)
        # byonic
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)[Byonic]': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract[Byonic]'] = all_combined_df.loc[all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]'])['Score[Byonic]'].idxmax(), 'Scan\r\nTime[Byonic]'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna[Byonic]'] = all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]'])['to_subtract[Byonic]'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif[Byonic]'] = (all_combined_df['Scan\r\nTime[Byonic]'] - all_combined_df['to_subtract_fillna[Byonic]']).abs() # get absolute dif
        all_combined_df['rt_dif_reset[Byonic]'] = [0 if dif < byonic_rt else dif for dif in all_combined_df['rt_dif[Byonic]']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        byonic_psm = all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset[Byonic]'])['N-site(SequonBased)[Byonic]'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('N-site(SequonBased)[Byonic]') + 1 , 'PSM[Byonic]', byonic_psm , True)
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == 'N/A'), 'PSM[Byonic]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM[Byonic]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == '-1'), 'PSM[Byonic]'] = -1 # if site is '-1', then psm set to -1
        # pglyco
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'int'})
        all_combined_df = all_combined_df.astype({'ProSites[pGlyco]': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract[pGlyco]'] = all_combined_df.loc[all_combined_df.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'])['GlyScore[pGlyco]'].idxmax(), 'RT[pGlyco]'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna[pGlyco]'] = all_combined_df.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]'])['to_subtract[pGlyco]'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif[pGlyco]'] = (all_combined_df['RT[pGlyco]'] - all_combined_df['to_subtract_fillna[pGlyco]']).abs() # get absolute dif
        all_combined_df['rt_dif_reset[pGlyco]'] = [0 if dif < pglyco_rt else dif for dif in all_combined_df['rt_dif[pGlyco]']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        pglyco_psm = all_combined_df.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset[pGlyco]'])['ProSites[pGlyco]'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('ProSites[pGlyco]') + 1 , 'PSM[pGlyco]', pglyco_psm , True)
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == 'N/A'), 'PSM[pGlyco]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[pglyco_belowthreshold_id, 'PSM[pGlyco]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['ProSites[pGlyco]'] == '-1'), 'PSM[pGlyco]'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['to_subtract[Byonic]', 'to_subtract_fillna[Byonic]', 'rt_dif[Byonic]', 'rt_dif_reset[Byonic]', 'to_subtract[pGlyco]', 'to_subtract_fillna[pGlyco]', 'rt_dif[pGlyco]', 'rt_dif_reset[pGlyco]'], axis = 1) # drop the intermediate cols

        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_All.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        if 'Pair[Byonic]' in all_combined_df.columns.tolist():
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
            pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_Pair.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[Byonic]' in col or '[Byos]' in col]
        pglyco_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[pGlyco]' in col]
        simple_byonicbyos = all_forsimple_df.loc[byonic_colored_id, byonicbyos_col_range] # extract colored (pass threshold) rows
        simple_byonicbyos = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # for simplified, n-site must exist
        simple_pglyco = all_forsimple_df.loc[pglyco_colored_id, pglyco_col_range] # extract colored (pass threshold) rows
        simple_pglyco = simple_pglyco[(simple_pglyco['ProSites[pGlyco]'] != 'N/A')&(simple_pglyco['ProSites[pGlyco]'] != '-1')] # for simplified, n-site must exist
        simple_pglyco = simple_pglyco.loc[simple_pglyco['FragmentType[pGlyco]'] == 'hcd'] # pglyco etd has the same values as hcd, so only let hcd enter unique file
        # get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos = simple_byonicbyos.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        simple_pglyco = simple_pglyco.groupby(['ProSites[pGlyco]', 'PrecursorMZ[pGlyco]','GlycanComposition_ByonicStyle[pGlyco]', 'Peptide[pGlyco]', 'rt_dif_reset[pGlyco]'], as_index=False).apply(lambda x: x.loc[x['GlyScore[pGlyco]'].idxmax()]).reset_index(drop=True)

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan
        simple_byonicbyos = simple_byonicbyos.set_index('Scan')
        simple_pglyco = simple_pglyco.set_index('Scan')
        a1, a2 = simple_byonicbyos.align(simple_pglyco, join = 'outer', axis = 0) # row: a1 = a2
        id_df = pd.concat([a1,a2], axis = 1)
        id_df.index.name = 'Scan'
        id_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        id_df = id_df.fillna(-1)
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df['PepMS[Byonic]'] = id_df['Calc.\r\nMH[Byonic]'] - id_df['glycan_mods_sum[Byonic]']
        id_df.loc[(id_df['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(id_df, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
        id_df['ProSites[pGlyco]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['ProSites[pGlyco]'].tolist()] # convert str back to int & tuple
        
        id_df.loc[(id_df['N-site(SequonBased)[Byonic]'] == -1), 'mock_site[Byonic]'] = id_df['ProSites[pGlyco]']
        id_df.loc[(id_df['N-site(SequonBased)[Byonic]'] != -1), 'mock_site[Byonic]'] = id_df['N-site(SequonBased)[Byonic]']
        # drop intermediate cols
        id_df = id_df.drop(['to_subtract[Byonic]', 'to_subtract_fillna[Byonic]', 'rt_dif[Byonic]', 'rt_dif_reset[Byonic]', 'to_subtract[pGlyco]', 'to_subtract_fillna[pGlyco]', 'rt_dif[pGlyco]', 'rt_dif_reset[pGlyco]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        glycansource(id_df, 'Scan')
        move_df(id_df, 'mock_site[Byonic]', 'GlycanSource')
        col_order = id_df.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df = sort_intstr_col(id_df, 'Charge[pGlyco]')
        print('Done sorting by Charge[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PrecursorMH[pGlyco]')
        print('Done sorting by PrecursorMH[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PeptideMH[pGlyco]')
        print('Done sorting by PeptideMH[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'GlycanComposition_ByonicStyle[pGlyco]')
        print('Done sorting by GlycanComposition_ByonicStyle[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Peptide[pGlyco]')
        print('Done sorting by Peptide[pGlyco] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Glycans[Byonic]')
        print('Done sorting by Glycans[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'mock_site[Byonic]')
        print('Done sorting by mock_site[Byonic] (added during processing)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df['Glycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df['Glycans[Byonic]']]
        id_df['GlycanComposition_ByonicStyle[pGlyco]'] = [int(i) if i == '-1' else i for i in id_df['GlycanComposition_ByonicStyle[pGlyco]']]
        id_df = id_df[col_order]
        threshold_masks_colorind(id_df)
        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        id_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_UniquePep.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')
    
    elif byonicfile != '' and byosfile == '' and pglycofile == '': # only byonic
        print('----- Only Byonic file is detected. Start the corresponding processing. -----\n')
        # change the variable name
        all_combined_df = byonic_df
        # make sure scan is at the start
        all_combined_df = all_combined_df.set_index('Scan')
        all_combined_df.reset_index(level=0, inplace=True)
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        # analyze PSM (need to pass threshold)
        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind)) # byonic pass threshold
        byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_colored_id]
        
        # using groupby size function to count psm & add psm columns (new criterion: RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif)
        # byonic
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract'] = all_combined_df.loc[all_combined_df.groupby(['N-site(SequonBased)', 'Calc.\r\nm/z','Glycans', 'PureSequence'])['Score'].idxmax(), 'Scan\r\nTime'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna'] = all_combined_df.groupby(['N-site(SequonBased)', 'Calc.\r\nm/z','Glycans', 'PureSequence'])['to_subtract'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif'] = (all_combined_df['Scan\r\nTime'] - all_combined_df['to_subtract_fillna']).abs() # get absolute dif
        all_combined_df['rt_dif_reset'] = [0 if dif < byonic_rt else dif for dif in all_combined_df['rt_dif']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        byonic_psm = all_combined_df.groupby(['N-site(SequonBased)', 'Calc.\r\nm/z','Glycans', 'PureSequence', 'rt_dif_reset'])['N-site(SequonBased)'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('N-site(SequonBased)') + 1 , 'PSM', byonic_psm , True)
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)'] == 'N/A'), 'PSM'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)'] == '-1'), 'PSM'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['to_subtract', 'to_subtract_fillna', 'rt_dif', 'rt_dif_reset'], axis = 1) # drop the intermediate cols
        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_All.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        if 'Pair' in all_combined_df.columns.tolist():
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair'].str.contains('pair')]
            pair_df = pair_df[pair_df.duplicated(subset=['Pair'], keep=False)] # only paired data included in _Pair
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_Pair.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list
        simple_byonicbyos = all_forsimple_df.loc[byonic_colored_id, :] # extract colored (pass threshold) rows
        simple_byonicbyos = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)'] != '-1')] # for simplified, n-site must exist
        
        # get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos = simple_byonicbyos.groupby(['N-site(SequonBased)', 'Calc.\r\nm/z','Glycans', 'PureSequence', 'rt_dif_reset'], as_index=False).apply(lambda x: x.loc[x['Score'].idxmax()]).reset_index(drop=True)
        
        # change variable name
        id_df = simple_byonicbyos
        
        # make sure 'Scan' is the 1st col
        id_df = id_df.set_index('Scan')
        id_df.reset_index(level=0, inplace=True)
        
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df['Mods\r\n(variable)'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df['glycan_mods_sum'] = mods_nglycan_sum
        id_df['PepMS'] = id_df['Calc.\r\nMH'] - id_df['glycan_mods_sum']
        id_df.loc[(id_df['glycan_mods_sum'] == -1), 'PepMS'] = -1
        move_df(id_df, 'PepMS', 'Fragment\r\nType')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df['N-site(SequonBased)'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['N-site(SequonBased)'].tolist()] # convert str back to int & tuple
        # drop intermediate cols
        id_df = id_df.drop(['to_subtract', 'to_subtract_fillna', 'rt_dif', 'rt_dif_reset', 'glycan_mods_sum'], axis = 1) # drop the intermediate cols
        col_order = id_df.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df = sort_intstr_col(id_df, 'z')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Calc.\r\nMH')
        print('Done sorting by Calc.MH --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PepMS')
        print('Done sorting by PepMS --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Glycans')
        print('Done sorting by Glycans --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PureSequence')
        print('Done sorting by PureSequence --> ', end = '')
        id_df = sort_intstr_col(id_df, 'N-site(SequonBased)')
        print('Done sorting by N-site(SequonBased)')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df['Glycans'] = [int(i) if i == '-1' else i for i in id_df['Glycans']]
        id_df = id_df[col_order]
        threshold_masks_colorind(id_df)
        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        id_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_UniquePep.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')
    elif byonicfile == '' and byosfile == '' and pglycofile != '': # only pglyco
        print('----- Only pGlyco file is detected. Start the corresponding processing. -----\n')
        # change the variable name
        all_combined_df = pglyco_df
        # make sure scan is at the start
        all_combined_df = all_combined_df.set_index('Scan')
        all_combined_df.reset_index(level=0, inplace=True)
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        # analyze PSM (need to pass threshold)
        pglyco_colored_id = list(set(lightblue_ind + normalblue_ind)) # pglyco pass threshold
        pglyco_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in pglyco_colored_id]
        # using groupby size function to count psm & add psm columns (new criterion: RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif)
        # pglyco
        all_combined_df = all_combined_df.astype({'ProSites': 'int'})
        all_combined_df = all_combined_df.astype({'ProSites': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract'] = all_combined_df.loc[all_combined_df.groupby(['ProSites', 'PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide'])['GlyScore'].idxmax(), 'RT'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna'] = all_combined_df.groupby(['ProSites', 'PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide'])['to_subtract'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif'] = (all_combined_df['RT'] - all_combined_df['to_subtract_fillna']).abs() # get absolute dif
        all_combined_df['rt_dif_reset'] = [0 if dif < pglyco_rt else dif for dif in all_combined_df['rt_dif']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        pglyco_psm = all_combined_df.groupby(['ProSites', 'PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide', 'rt_dif_reset'])['ProSites'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('ProSites') + 1 , 'PSM', pglyco_psm , True)
        all_combined_df.loc[(all_combined_df['ProSites'] == 'N/A'), 'PSM'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[pglyco_belowthreshold_id, 'PSM'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['ProSites'] == '-1'), 'PSM'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['to_subtract', 'to_subtract_fillna', 'rt_dif', 'rt_dif_reset', 'to_subtract', 'to_subtract_fillna', 'rt_dif', 'rt_dif_reset'], axis = 1) # drop the intermediate cols

        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_All.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        simple_pglyco = all_forsimple_df.loc[pglyco_colored_id, :] # extract colored (pass threshold) rows
        simple_pglyco = simple_pglyco[(simple_pglyco['ProSites'] != 'N/A')&(simple_pglyco['ProSites'] != '-1')] # for simplified, n-site must exist
        simple_pglyco = simple_pglyco.loc[simple_pglyco['FragmentType[pGlyco]'] == 'hcd'] # pglyco etd has the same values as hcd, so only let hcd enter unique file
        # get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_pglyco = simple_pglyco.groupby(['ProSites', 'PrecursorMZ','GlycanComposition_ByonicStyle', 'Peptide', 'rt_dif_reset'], as_index=False).apply(lambda x: x.loc[x['GlyScore'].idxmax()]).reset_index(drop=True)

        # set scan to index & concat simple_byonicbyos & simple_pglyco by scan
        simple_pglyco = simple_pglyco.set_index('Scan')
        simple_pglyco.reset_index(level=0, inplace=True)
        # change variable name
        id_df = simple_pglyco
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df['ProSites'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['ProSites'].tolist()] # convert str back to int & tuple
        # drop intermediate cols
        id_df = id_df.drop(['to_subtract', 'to_subtract_fillna', 'rt_dif', 'rt_dif_reset'], axis = 1) # drop the intermediate cols
        col_order = id_df.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df = sort_intstr_col(id_df, 'Charge')
        print('Done sorting by Charge --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PrecursorMH')
        print('Done sorting by PrecursorMH --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PeptideMH')
        print('Done sorting by PeptideMH --> ', end = '')
        id_df = sort_intstr_col(id_df, 'GlycanComposition_ByonicStyle')
        print('Done sorting by GlycanComposition_ByonicStyle --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Peptide')
        print('Done sorting by Peptide --> ', end = '')
        id_df = sort_intstr_col(id_df, 'ProSites')
        print('Done sorting by ProSites')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df['GlycanComposition_ByonicStyle'] = [int(i) if i == '-1' else i for i in id_df['GlycanComposition_ByonicStyle']]
        id_df = id_df[col_order]
        threshold_masks_colorind(id_df)
        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        id_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_UniquePep.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')
    elif byonicfile == '' and byosfile != '' and pglycofile == '': # only byos 
        print('----- Only Byos file is detected. Please at least submit either Byonic or pGlyco file for further processing. -----\n')
    elif byonicfile == '' and byosfile != '' and pglycofile == '': # byos + pglyco 
        print('----- Byos & pGlyco files are detected. Please submit Byonic file for further processing. -----\n')
    elif byonicfile != '' and byosfile != '' and pglycofile == '': # byonic + byos 
        print('----- Start combining Byonic & Byos. -----\n')
        # combined data based on 'Scan'
        byonic_scanasid = byonic_df.copy()
        new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
        byonic_scanasid.columns = new_byonic_col
        byos_scanasid = byos_df.copy()
        new_byos_col = [n + '[Byos]' if n != 'Scan Number(s)\r\n(Posit)' else n for n in byos_scanasid.columns]
        byos_scanasid.columns = new_byos_col
        byonic_scanasid = byonic_scanasid.set_index('Scan')
        byos_scanasid = byos_scanasid.set_index('Scan Number(s)\r\n(Posit)')
        # align scan & concat (all align on row to make row number all the same)
        a1, a2 = byonic_scanasid.align(byos_scanasid, join = 'outer', axis = 0) # row: a1 = a2
        all_combined_df = pd.concat([a1,a2], axis = 1)
        all_combined_df.index.name = 'Scan'
        all_combined_df.reset_index(level=0, inplace=True)
        # change all nan to blank -1
        all_combined_df = all_combined_df.fillna(-1)
        ## result post-processing 
        print('\nCombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))
        ## style apply for excel export
        # color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])
        # color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco; #FFB6C1 -> light pink for byos w/ dif calm ^ seq from byonic; #FF1493 -> deep pink for byos w/ dif calm & seq from byonic; #FFFF00 -> yellow for byos & byonic all the same
        threshold_masks_colorind(all_combined_df)
        # analyze PSM (need to pass threshold)
        byonic_colored_id = list(set(lightgreen_ind + normalgreen_ind)) # byonic pass threshold
        byonic_belowthreshold_id = [i for i in all_combined_df.index.tolist() if i not in byonic_colored_id]
        # using groupby size function to count psm & add psm columns (new criterion: RT/scantime dif < 3 min (180s). is regarded as the same, >= 3 min. should be seen as dif)
        # byonic & byos
        all_combined_df = all_combined_df.astype({'N-site(SequonBased)[Byonic]': 'str'}) # convert list & tuple to str for later groupby function
        all_combined_df['to_subtract[Byonic]'] = all_combined_df.loc[all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]'])['Score[Byonic]'].idxmax(), 'Scan\r\nTime[Byonic]'] # get the rt of highest score as ref for rt_dif < 3 min.
        all_combined_df['to_subtract_fillna[Byonic]'] = all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]'])['to_subtract[Byonic]'].transform(lambda x: x.fillna(x.mean())) # fillna w/ rt of highest score
        all_combined_df['rt_dif[Byonic]'] = (all_combined_df['Scan\r\nTime[Byonic]'] - all_combined_df['to_subtract_fillna[Byonic]']).abs() # get absolute dif
        all_combined_df['rt_dif_reset[Byonic]'] = [0 if dif < byonic_rt else dif for dif in all_combined_df['rt_dif[Byonic]']] # if rt_dif < 3, then reset the dif to 0 -> seen as the same
        byonic_psm = all_combined_df.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset[Byonic]'])['N-site(SequonBased)[Byonic]'].transform('size')
        all_combined_df.insert(all_combined_df.columns.get_loc('N-site(SequonBased)[Byonic]') + 1 , 'PSM[Byonic]', byonic_psm , True)
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == 'N/A'), 'PSM[Byonic]'] = 'N/A' # if n-site is 'N/A', do not count psm
        all_combined_df.loc[byonic_belowthreshold_id, 'PSM[Byonic]'] = 'N/A' # do not count psm for rows below threshold
        all_combined_df.loc[(all_combined_df['N-site(SequonBased)[Byonic]'] == '-1'), 'PSM[Byonic]'] = -1 # if site is '-1', then psm set to -1
        all_forsimple_df = all_combined_df.copy() # to retain rt_dif_reset for simple df groupy and get unique row
        all_combined_df = all_combined_df.drop(['to_subtract[Byonic]', 'to_subtract_fillna[Byonic]', 'rt_dif[Byonic]', 'rt_dif_reset[Byonic]'], axis = 1) # drop the intermediate cols

        print('\n----- Exporting "_All" file... This may take some time, please wait. -----\n')
        all_combined_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_All.xlsx', index = False)  
        print('\n----- "_All" file exported. -----\n')
        
        if 'Pair[Byonic]' in all_combined_df.columns.tolist():
            # ONLY FOR HCD/ETD FILES: prepare '_Pair' file (as long as byonic passes threshold & have pair)
            all_combined_df = all_combined_df.astype({'Pair[Byonic]': 'str'})
            pair_df = all_combined_df.loc[byonic_colored_id]
            pair_df = pair_df[pair_df['Pair[Byonic]'].str.contains('pair')]
            pair_df = pair_df[pair_df.duplicated(subset=['Pair[Byonic]'], keep=False)] # only paired data included in _Pair
            threshold_masks_colorind(pair_df)
            print('\n----- Exporting "_Pair" file... This may take some time, please wait. -----\n')
            pair_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_Pair.xlsx', index = False)  
            print('\n----- "_Pair" file exported. -----\n')
        else:
            pass

        print('----- Start preparing simplified version. -----\n')
        # extract rows included in colored indices list separately
        byonicbyos_col_range = ['Scan'] + [col for col in all_forsimple_df.columns.tolist() if '[Byonic]' in col or '[Byos]' in col]
        simple_byonicbyos = all_forsimple_df.loc[byonic_colored_id, byonicbyos_col_range] # extract colored (pass threshold) rows
        simple_byonicbyos = simple_byonicbyos[(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != 'N/A')&(simple_byonicbyos['N-site(SequonBased)[Byonic]'] != '-1')] # for simplified, n-site must exist
        
        # get unique calc.m/z by groupby & idxmax function to find highest score within each gp and preserve (remember to split the simple_df into byonicbyos & pglyco parts to avoid dropping non-target rows)
        simple_byonicbyos = simple_byonicbyos.groupby(['N-site(SequonBased)[Byonic]', 'Calc.\r\nm/z[Byonic]','Glycans[Byonic]', 'PureSequence[Byonic]', 'rt_dif_reset[Byonic]'], as_index=False).apply(lambda x: x.loc[x['Score[Byonic]'].idxmax()]).reset_index(drop=True)
        
        # make sure scan is the 1st col
        simple_byonicbyos = simple_byonicbyos.set_index('Scan')
        simple_byonicbyos.reset_index(level=0, inplace=True)
        # change variable name
        id_df = simple_byonicbyos
        # extract nglycan numbers in Mods & use it to calculate byonic pep mass 
        mods_nglycan_sum = [sum([ast.literal_eval(v) for v in lst]) if type(lst) == list else -1 for lst in id_df['Mods\r\n(variable)[Byonic]'].str.findall(r'Glycan / (\d+.\d+)').tolist()]
        id_df['glycan_mods_sum[Byonic]'] = mods_nglycan_sum
        id_df['PepMS[Byonic]'] = id_df['Calc.\r\nMH[Byonic]'] - id_df['glycan_mods_sum[Byonic]']
        id_df.loc[(id_df['glycan_mods_sum[Byonic]'] == -1), 'PepMS[Byonic]'] = -1
        move_df(id_df, 'PepMS[Byonic]', 'Fragment\r\nType[Byonic]')
        # ast.literaleval byonicbyos n-site & sorting: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic clam/z -> pglcyo calm/z
        id_df['N-site(SequonBased)[Byonic]'] = [ast.literal_eval(i) if type(i) == str and '(' not in i else i for i in id_df['N-site(SequonBased)[Byonic]'].tolist()] # convert str back to int & tuple
                
        # drop intermediate cols
        id_df = id_df.drop(['to_subtract[Byonic]', 'to_subtract_fillna[Byonic]', 'rt_dif[Byonic]', 'rt_dif_reset[Byonic]', 'glycan_mods_sum[Byonic]'], axis = 1) # drop the intermediate cols
        
        col_order = id_df.columns.tolist()
        # sorting order: byonic site -> pglyco site -> byonic peptide mass -> pglyco peptide mass -> byonic glycans -> pglyco glycans in byonic style -> byonic calm/z -> pglcyo calm/z => all of this can be acheive with sort_values to multi cols.

        id_df = sort_intstr_col(id_df, 'z[Byonic]')
        print('Done sorting by z[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Calc.\r\nMH[Byonic]')
        print('Done sorting by Calc.MH[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PepMS[Byonic]')
        print('Done sorting by PepMS[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'Glycans[Byonic]')
        print('Done sorting by Glycans[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'PureSequence[Byonic]')
        print('Done sorting by PureSequence[Byonic] --> ', end = '')
        id_df = sort_intstr_col(id_df, 'N-site(SequonBased)[Byonic]')
        print('Done sorting by N-site(SequonBased)[Byonic]')

        # convert '-1' back to int -1 for correct glycansource analysis
        id_df['Glycans[Byonic]'] = [int(i) if i == '-1' else i for i in id_df['Glycans[Byonic]']]
        
        id_df = id_df[col_order]
        threshold_masks_colorind(id_df)
        print('\n----- Exporting "_UniquePep" file... This may take some time, please wait. -----\n')
        id_df.style.apply(bg_color, axis=None).to_excel(f'{date}_{filename}_UniquePep.xlsx', index = False)  
        print('\n----- "_UniquePep" file exported. -----\n')
        print('----- Start preparing quantification file. -----\n')

        # quantification only apply to single sites (quant_df offers important info. for later multiIndex df construction)
        quant = id_df[id_df['N-site(SequonBased)[Byonic]'].apply(lambda x: isinstance(x, int))]
        drop_ind = quant.loc[(quant['Fragment\r\nType[Byonic]'] == 'ethcd') & ((quant['Score[Byonic]'] <= 200)|(quant['PEP\r\n2D[Byonic]'].abs() >= 0.001)) & ((quant['Pair[Byonic]'] == -1)|(quant['Pair[Byonic]'] == 'N/A'))].index.tolist()
        ind = [i for i in quant.index.tolist() if i not in drop_ind]
        quant = quant.loc[ind] # hcd all in, if byonic etd pass score & pep2d all in, otherwise in with pair
        # normalize by sum of xicauc/ int/ mono/ iso respectively
        total_xicauc = quant.loc[quant['XIC\r\nAUC[Byos]'] != -1, 'XIC\r\nAUC[Byos]'].sum()
        print('total_xicauc:%s'%total_xicauc)
        total_int = quant.loc[quant['Apex Int.\r\n(Posit)[Byos]'] != -1, 'Apex Int.\r\n(Posit)[Byos]'].sum()
        print('total_int:%s'%total_int)
        total_mono = quant.loc[quant['MonoArea[pGlyco]'] != -1, 'MonoArea[pGlyco]'].sum()
        print('total_mono:%s'%total_mono)
        total_iso = quant.loc[quant['IsotopeArea[pGlyco]'] != -1, 'IsotopeArea[pGlyco]'].sum()
        print('total_iso:%s'%total_iso)
        # avoid adding values from below-threshold rows: if deepgreen -> change value to -1, if deepblue -> change value to -1.(-1 to make lambda function easier to write)
        threshold_masks_colorind(quant)
        quant.loc[deepgreen_ind, ['MonoArea[pGlyco]', 'IsotopeArea[pGlyco]']] = -1
        quant.loc[deepblue_ind, ['XIC\r\nAUC[Byos]', 'Apex Int.\r\n(Posit)[Byos]']] = -1
        # convert all the 'N/A' in byonic glycans to 'Unoccupied'
        quant.loc[(quant['Glycans[Byonic]'] == 'N/A'), ['Glycans[Byonic]']] = 'Unoccupied'
        # using groupby transform sum function to add 8 cols (byos xicauc/byos area int/pglycomono/pglycoisotope) recording summed values & normalized values (same site & same glycan & seq can be dif)
        quant['e_sum_XIC\r\nAUC[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['f_sum_Apex Int.\r\n(Posit)[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x != -1].sum())
        quant['g_sum_MonoArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['h_sum_IsotopeArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x != -1].sum())
        quant['a_norm_XIC\r\nAUC[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['XIC\r\nAUC[Byos]'].transform(lambda x: x[x != -1].sum()/total_xicauc)
        quant['b_norm_Apex Int.\r\n(Posit)[Byos]'] = quant.groupby(['N-site(SequonBased)[Byonic]', 'Glycans[Byonic]'])['Apex Int.\r\n(Posit)[Byos]'].transform(lambda x: x[x != -1].sum()/total_int)
        quant['c_norm_MonoArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['MonoArea[pGlyco]'].transform(lambda x: x[x != -1].sum()/total_mono)
        quant['d_norm_IsotopeArea[pGlyco]'] = quant.groupby(['ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]'])['IsotopeArea[pGlyco]'].transform(lambda x: x[x != -1].sum()/total_iso)
        # since it's possible to have real 0 from calculation, change the real absent data back to -1 in sum_ & norm_
        quant.loc[quant['MonoArea[pGlyco]'] == -1, ['g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']] = -1
        quant.loc[quant['XIC\r\nAUC[Byos]'] == -1, ['e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']] = -1 
        # extract the needed cols only & split byonicbyos/pglyco, drop all -1 rows, glycans as index to outer union concat data again
        quant_bb = quant[['Scan', 'N-site(SequonBased)[Byonic]', 'Glycans[Byonic]', 'MS2 Search\r\nAlias name[Byos]', 'XIC\r\nAUC[Byos]', 'Apex Int.\r\n(Posit)[Byos]', 'e_sum_XIC\r\nAUC[Byos]', 'f_sum_Apex Int.\r\n(Posit)[Byos]', 'a_norm_XIC\r\nAUC[Byos]', 'b_norm_Apex Int.\r\n(Posit)[Byos]']]
        quant_bb = quant_bb.rename(columns={'Scan': 'Scan[Byonic]'})
        print('quant_bb len: %s'%len(quant_bb))
        quant_p = quant[['Scan', 'ProSites[pGlyco]', 'GlycanComposition_ByonicStyle[pGlyco]', 'MonoArea[pGlyco]', 'IsotopeArea[pGlyco]', 'g_sum_MonoArea[pGlyco]', 'h_sum_IsotopeArea[pGlyco]', 'c_norm_MonoArea[pGlyco]', 'd_norm_IsotopeArea[pGlyco]']]
        quant_p = quant_p.rename(columns={'Scan': 'Scan[pGlyco]'})
        print('quant_p len: %s'%len(quant_p))
        quant_bb = quant_bb.loc[quant_bb['N-site(SequonBased)[Byonic]'] != -1]
        print('cleaned up quant_bb len: %s'%len(quant_bb))
        quant_p = quant_p.loc[quant_p['ProSites[pGlyco]'] != -1]
        print('cleaned up quant_p len: %s'%len(quant_p))
        quant_bb = quant_bb.set_index('Glycans[Byonic]')
        quant_p = quant_p.set_index('GlycanComposition_ByonicStyle[pGlyco]')
        a1, a2 = quant_bb.align(quant_p, join = 'outer', axis = 0) 
        quant_glycanid = pd.concat([a1,a2], axis = 1)
        quant_glycanid.index.name = 'Glycans ↓'
        quant_glycanid.reset_index(level=0, inplace=True)
        # add N(x) col
        nx = ['Unoccupied' if lst == [] else 'N(%s)'%(lst[0]) for lst in quant_glycanid['Glycans ↓'].str.findall(r'HexNAc\((\d+)\)').tolist()]
        quant_glycanid.insert(0 , 'N(x) ↓', nx , True)
        move_df(quant_glycanid, 'ProSites[pGlyco]', 'N-site(SequonBased)[Byonic]')
        move_df(quant_glycanid, 'Scan[Byonic]', 'N-site(SequonBased)[Byonic]')
        move_df(quant_glycanid, 'Scan[pGlyco]', 'ProSites[pGlyco]')
        # change all nan to blank -1
        quant_glycanid = quant_glycanid.fillna(-1)
        # quant_glycanid.to_excel('quant_glycanid.xlsx', index = False)
        print('----- Start summary table construction. -----\n')
        # start grouping: N(X) -> byonic glycans -> byonic sites -> .agg({all 8 cols, apply .mean() & .mean()/total XXX})
        quant = quant_glycanid.groupby(['N(x) ↓', 'Glycans ↓', 'N-site(SequonBased)[Byonic]', 'ProSites[pGlyco]']).agg({'e_sum_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'f_sum_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean() \
                                                                                              , 'g_sum_MonoArea[pGlyco]': lambda x: x.mean(), 'h_sum_IsotopeArea[pGlyco]': lambda x: x.mean() \
                                                                                              , 'a_norm_XIC\r\nAUC[Byos]': lambda x: x.mean(), 'b_norm_Apex Int.\r\n(Posit)[Byos]': lambda x: x.mean() \
                                                                                              , 'c_norm_MonoArea[pGlyco]': lambda x: x.mean(), 'd_norm_IsotopeArea[pGlyco]': lambda x: x.mean()})
                              
        # fix the str '10' sorted before '2' problem
        nx = list(set(nx))
        nx  = [int(re.findall(r'[0-9]+', i)[0]) if type(i) == str and i != 'Unoccupied' else i for i in nx]
        nx.sort(key=lambda v: (isinstance(v, str), v))
        new_nx = ['N(%s)'%(str(i)) if type(i) == int and i != -1 else i for i in nx]
        quant = quant.reindex(new_nx, level = 0)
        # unstack ms2 alias name, byonic n-sites, pglyco n-sites
        quant = quant.unstack(level=-1)
        quant = quant.unstack(level=-1)
        # multiIndex col swaplevel
        quant.columns = quant.columns.swaplevel(0, 2)
        quant.columns = quant.columns.swaplevel(0, 1)
        # sort is necessary for the right table structure (but since it will sort every level, so we need to manually adjust level3 back to the original order)
        quant.sort_index(axis=1, level=[0, 1, 2], inplace=True)
        quant.columns = quant.columns.set_names(['ProSites[pGlyco] →', 'N-site(SequonBased)[Byonic] →', 'Quant. →'])
        # delete a-h in Quant. (the alphabet is just for sorting)
        quant.columns.set_levels(['norm_XIC\r\nAUC[Byos]', 'norm_Apex Int.\r\n(Posit)[Byos]', 'norm_MonoArea[pGlyco]', 'norm_IsotopeArea[pGlyco]', 'sum_XIC\r\nAUC[Byos]', 'sum_Apex Int.\r\n(Posit)[Byos]', 'sum_MonoArea[pGlyco]', 'sum_IsotopeArea[pGlyco]'], level = 2, inplace=True)

        print('\n----- Exporting "_Quant" file... This may take some time, please wait. -----\n')
        # quant.style.background_gradient(cmap ='Reds').to_excel(f'{date}_{filename}_All.xlsx')  
        print('\n----- "_Quant" file exported. -----\n')
    # EXECUTION TIME
    print("\nAll tasks completed.\nExecution time: %.2f seconds"%(time.time() - start_time))
    
# RUN PROCESSING
main()

Original byonic columns:
Index(['PID', 'Prot._x000D_\nRank', 'Pos.', 'Sequence',
       'SequenceOriginalDebug', 'PeptideParseFriendly', 'Calc._x000D_\nMH',
       'Obs._x000D_\nm/z', 'Calc._x000D_\nm/z', 'z', 'Off-_x000D_\nBy-X',
       'ppm_x000D_\nerr.', 'Score', 'PEP_x000D_\n2D', 'Scan_x000D_\nTime',
       'Fragment_x000D_\nType', 'Glycans', 'Mods_x000D_\n(variable)', 'Scan #',
       'Cleavage', 'PEP_x000D_\n1D', '|Log Prob|', 'Delta_x000D_\nScore',
       'Delta Mod._x000D_\nScore', 'Obs._x000D_\nMH', 'Glycans_x000D_\nPos.',
       'Z-Score', 'Protein_x000D_\nName', 'Prot._x000D_\nId', 'QID',
       'Mods_x000D_\n(fixed)', '|LogProbOld|', 'Obs.M', 'Calc.M',
       'FDR_x000D_\n2D', 'FDR_x000D_\n1D', 'FDR uniq._x000D_\n2D',
       'FDR uniq._x000D_\n1D', 'q-value_x000D_\n2D', 'q-value_x000D_\n1D',
       'Comment'],
      dtype='object')

Fixed byonic columns:
Index(['PID', 'Prot.\r\nRank', 'Pos.', 'Sequence', 'SequenceOriginalDebug',
       'PeptideParseFriendly', 'Calc.\r\nMH',

<ipython-input-17-8e73e5d856e6>:615: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-17-8e73e5d856e6>:615: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-17-8e73e5d856e6>:615: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-17-8e73e5d856e6>:615: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.M', 'PureSequence'], keep=False)]
<ipython-input-1

----- Byonic data preprocessing completed. -----

Original pglyco columns:
Index(['GlySpec', 'PepSpec', 'RawName', 'Scan', 'RT', 'PrecursorMH',
       'PrecursorMZ', 'Charge', 'Rank', 'Peptide', 'Mod', 'PeptideMH',
       'Glycan(H,N,F,A)', 'GlycanComposition', 'PlausibleStruct', 'GlyID',
       'GlyFrag', 'GlyMass', 'GlySite', 'TotalScore', 'PepScore', 'GlyScore',
       'CoreMatched', 'MassDeviation', 'PPM', 'GlyIonRatio', 'byIonRatio',
       'czIonRatio', 'GlyDecoy', 'PepDecoy', 'IsSmallGlycan', 'GlycanPEP',
       'GlycanFDR', 'PeptidePEP', 'PeptideFDR', 'TotalFDR', 'Proteins',
       'Genes', 'ProSites', 'MonoArea', 'IsotopeArea', 'ETDScan',
       'LocalizedSiteGroups', 'LocalizedScore', 'LocalizedIonRatio',
       'PreLocalizedScore'],
      dtype='object')

Fixed pglyco columns:
Index(['GlySpec', 'PepSpec', 'RawName', 'Scan', 'RT', 'PrecursorMH',
       'PrecursorMZ', 'Charge', 'Rank', 'Peptide', 'Mod', 'PeptideMH',
       'Glycan(H,N,F,A)', 'GlycanComposition', 'PlausibleStru